In [1]:
import json, time, os
from elasticsearch import Elasticsearch

### Connecting to ElasticSearch instance

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

### Querying for Brexit Text

In [8]:
start = time.time()

# Defining Brexit search query 
search_object = {'query': {'match': {'text': 'Brexit'}}}

# Applying query term to news corpus indexed on ES
sch_obj = es.search(index="epic_text", body=search_object) #Increase returns with size parameter, default 10

print(time.time()-start)
sch_obj

0.016999006271362305


{'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1745, 'relation': 'eq'},
  'max_score': 8.370473,
  'hits': [{'_index': 'epic_text',
    '_type': '_doc',
    '_id': '151089',
    '_score': 8.370473,
    '_source': {'category': 'general',
     'url': 'http://www.telegraph.co.uk/business/2017/09/16/biggest-problem-soft-brexit-not-attainable/',
     'text': 'In the first of two extracts from their new book, Liam Halligan and Gerard Lyons say the commonly held belief that Britain would be better off inside the single market and customs union is misconceived\n\nThere has been much talk of “Hard Brexit” versus “Soft Brexit”. Such labels are ubiquitous during these Article 50 negotiations – used freely by the broadcast media – yet they are partisan and deeply misleading. Hard Brexit makes leaving the European Union sound extreme and damaging, suggesting isola\xadtion and a bleak economic future. Soft Brexit, 

In [4]:
# Grabbing the list containing queries 
hits = sch_obj['hits']['hits']

# Sample query text
hits[0]['_source']['text']

'In the first of two extracts from their new book, Liam Halligan and Gerard Lyons say the commonly held belief that Britain would be better off inside the single market and customs union is misconceived\n\nThere has been much talk of “Hard Brexit” versus “Soft Brexit”. Such labels are ubiquitous during these Article 50 negotiations – used freely by the broadcast media – yet they are partisan and deeply misleading. Hard Brexit makes leaving the European Union sound extreme and damaging, suggesting isola\xadtion and a bleak economic future. Soft Brexit, conversely, conveys a comfortable, ongoing relationship with the EU, with Britain still “part of the club”.\n\nLeaving the single market and the customs union isn’t Hard Brexit – even if the name is deliberately coined to sound painful. It is simply Brexit. Staying inside the EU’s two main legal constructs, meanwhile, isn’t a harmonious Soft Brexit. It amounts, instead, to a deliberate and cynical failure to implement the 2016 referendum 

In [5]:
hit_text = []

# Iterate through list and extract text
for hit in hits:
    hit_text.append(hit['_source']['text'])
    
len(hit_text)

10

### Performing Sentiment Analysis

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

# Initialize Vader analyser
analyser = SentimentIntensityAnalyzer()

sentiment= []

# Perform sentiment analysis on every text result and append to new list
for number, i in enumerate(hit_text):
    sentiment.append((number, analyser.polarity_scores(i)))

In [10]:
hit_text[0]

'In the first of two extracts from their new book, Liam Halligan and Gerard Lyons say the commonly held belief that Britain would be better off inside the single market and customs union is misconceived\n\nThere has been much talk of “Hard Brexit” versus “Soft Brexit”. Such labels are ubiquitous during these Article 50 negotiations – used freely by the broadcast media – yet they are partisan and deeply misleading. Hard Brexit makes leaving the European Union sound extreme and damaging, suggesting isola\xadtion and a bleak economic future. Soft Brexit, conversely, conveys a comfortable, ongoing relationship with the EU, with Britain still “part of the club”.\n\nLeaving the single market and the customs union isn’t Hard Brexit – even if the name is deliberately coined to sound painful. It is simply Brexit. Staying inside the EU’s two main legal constructs, meanwhile, isn’t a harmonious Soft Brexit. It amounts, instead, to a deliberate and cynical failure to implement the 2016 referendum 

In [11]:
sentiment[0]

(0, {'neg': 0.106, 'neu': 0.816, 'pos': 0.078, 'compound': -0.5926})

In [14]:
hit_text[9]

'Listen to the Telegraph\'s weekly Brexit podcast, hosted by Chief Political Correspondent Christopher \'Chopper\' Hope and recorded at the historic Red Lion pub, a stone\'s throw from the Houses of Parliament.\n\nIn the\xa0second episode\xa0of the new season, Chopper\'s Brexit Podcast\xa0hears from Nick Tyrone, author of "Apocalypse Delayed"\xa0on how a clean Brexit will help the Tories win the next general election. Also the\xa0Liberal Democrats\'\xa0Brexit spokesman Tom Brake\xa0and\xa0Tory\xa0Brexiteer\xa0Michael Tomlinson\xa0debate the usefulness of a second referendum, and\xa0Solicitor General Robert Buckland, the Government minister who has to steer the EU Withdrawal Bill through Parliament,\xa0discusses\xa0whether 19,000 EU regulations can be repatriated to UK law by March 2019.\n\nRevisit the previous series of Chopper\'s Brexit Podcast here.\n\nTo listen offline, subscribe to the podcast:'

In [15]:
sentiment[9]

(9, {'neg': 0.024, 'neu': 0.866, 'pos': 0.11, 'compound': 0.9001})

In [16]:
snippet = ' It amounts, instead, to a deliberate and cynical failure to implement the 2016 referendum result.'
analyser.polarity_scores(snippet)

{'neg': 0.33, 'neu': 0.67, 'pos': 0.0, 'compound': -0.7096}